In [21]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from keras.models import Sequential
from keras import layers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pandas as pd
from Multimodal_baseline_Functions_No_Ran import *
from sklearn import model_selection, naive_bayes, svm

In [23]:
# Dividing data in test, train, validation
training_DF, testing_DF, validation_DF = preprocess_text(Training_path,Validation_path, Testing_path)

In [3]:
# # df = pd.read_csv('first_try.csv', names=['image_name','sentence', 'label'], sep='\t')
# df = pd.read_csv("E:\\MSc DA\\Sem 2\\Project\\Code and Docmentation\\Final Dataset\\sent_process_DF_2.csv", sep='\t')
# df = df.iloc[:,1:]

In [26]:
training_DF.head()

,image_name,sentence,label
0,LJ3r8Gy.jpg.png,official bernie sanders drinking game every ti...,0
1,qDnIIHA.png,228 pm wall inside nazi gas chamber dwebs demo...,1
2,1JQk5NF.png,shit waddup bernie sanders com,1
3,iMMNq.png,mitt romney worst republican country putup bar...,0
4,jAi3iI1.png,anonymous id duqda1io 08 05 16 fri 163248 8423...,0


In [34]:
# taking in array of sentences
sentences_train=training_DF['sentence'].values
sentences_test = testing_DF['sentence'].values
sentences_val = validation_DF['sentence'].values
# Instance of encoder
Encoder = LabelEncoder()
# taking in array of labels
y_train=training_DF['label'].values
y_train = Encoder.fit_transform(y_train)

y_test=testing_DF['label'].values
y_test = Encoder.fit_transform(y_test)

y_val=validation_DF['label'].values
y_val = Encoder.fit_transform(y_val)

In [35]:
# Testing
print(sentences_train, sentences_test, y_train, y_test)

['official bernie sanders drinking game every time bernster mentions free government program chug somebody else beer'
 '228 pm wall inside nazi gas chamber dwebs democrat mentions gun control remember picture'
 'shit waddup bernie sanders com'
 'mitt romney worst republican country putup barack obama endorse mitt romney'
 'anonymous id duqda1io 08 05 16 fri 163248 84231555 84231620 84231654 84231659 84231689 84231698 2 84231735 84231790 84231867 84231910 84232021 84232120 84232180 84232228 84232342 84232377 84232671 84232765 84232970 file image jpg 92 kb 740547 repeating digits pass away getting witness trump vs hillary debate'
 'kle watching dem debate hell yeah bernie hillary avoiding questions stahp hillary fucking stop swear fucking god hillary answer god damn question stop avoiding topic actually fucking debate ali'
 'trump isanasshole hillary isaliar iiwishicould vote negroagain'
 'nt cop problem america problem cops sure black america cop problem cops neighborhood neighborhood l

In [39]:
# instance of count vectoriser
vectorizer = CountVectorizer()
# converting sentences into vectors
vectorizer.fit(sentences_train)
# Creating training and test vectors
X_train = vectorizer.transform(sentences_train)
X_test  = vectorizer.transform(sentences_test)
X_val = vectorizer.transform(sentences_val)

In [40]:
#Baseline Logistic Regression
# Instance of classifier
classifier = LogisticRegression()
# Training classifier 
classifier.fit(X_train, y_train)
# Performace Metric Accuracy 
score = classifier.score(X_val, y_val)
print("Accuracy:", score)

Accuracy: 0.6510067114093959


f:\anaconda\envs\exp_env\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [41]:
y_pred = classifier.predict(X_test)

In [18]:
from sklearn.metrics import precision_recall_fscore_support

In [42]:
precision_recall_fscore_support(y_test, y_pred, beta=1.0, labels=None, pos_label=1, average='binary')

(0.5853658536585366, 0.4067796610169492, 0.48, None)

In [43]:
# fit the training dataset on the NB classifier
from sklearn import naive_bayes
Naive = naive_bayes.MultinomialNB()
Naive.fit(X_train,y_train)
# predict the labels on validation dataset
predictions_NB = Naive.predict(X_test)
# Use accuracy_score function to get the accuracy
score = Naive.score(X_val, y_val)
print("Naive Bayes Accuracy Score -> ",score)

Naive Bayes Accuracy Score ->  0.6241610738255033


In [44]:
precision_recall_fscore_support(y_test, predictions_NB, beta=1.0, labels=None, pos_label=1, average='binary')

(0.5294117647058824, 0.4576271186440678, 0.49090909090909096, None)

In [56]:
# Testing
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(sentences_train)
X_train = tokenizer.texts_to_sequences(sentences_train)
X_test = tokenizer.texts_to_sequences(sentences_test)
X_val = tokenizer.texts_to_sequences(sentences_val)
vocab_size = len(tokenizer.word_index) + 1
print(sentences_train[2])
print(X_train[2])

shit waddup bernie sanders com
[108, 325, 2, 7, 68]


In [57]:
# Assigning limits to vector length
maxlen = 100
# Padding X_train and X_test with 0 to match the length
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)
X_val = pad_sequences(X_val, padding='post', maxlen=maxlen)
print(X_train[0, :])

[ 544    2    7 1422  796  321   67 1423  797   78   79  545 1424  798
  119  322    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]


In [58]:
# NN Model
# Creating word embeddings
embedding_dim = 50
# Initiating sequential model
model = Sequential()
# Adding embedding layer
model.add(layers.Embedding(input_dim=vocab_size, 
                           output_dim=embedding_dim, 
                           input_length=maxlen))
model.add(layers.Flatten())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 100, 50)           219000    
_________________________________________________________________
flatten_4 (Flatten)          (None, 5000)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 10)                50010     
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 11        
Total params: 269,021
Trainable params: 269,021
Non-trainable params: 0
_________________________________________________________________


In [59]:
# Training the given model 
history = model.fit(X_train, y_train,
                    epochs=20,
                    verbose=False,
                    validation_data=(X_test, y_test),
                    batch_size=10)
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_val, y_val, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Training Accuracy: 0.9910
Testing Accuracy:  0.5839


In [63]:
y_pred = np.round(model.predict(X_test))

In [64]:
precision_recall_fscore_support(y_test, y_pred, beta=1.0, labels=None, pos_label=1, average='binary')

(0.47761194029850745, 0.5423728813559322, 0.5079365079365079, None)